### Case #01: The Overtime King
### One employee in 'Production' is logging 60 hours of overtime every single week, always on the same shift.
#### Objectives:
* Find employees in the 'Production' Department.
* Check the HumanResources.EmployeePayHistory for employees with a very high Rate. This is a trick; overtime isn't explicitly stored.
* Real Objective: Find the employee in the 'Production' department with the highest VacationHours and SickLeaveHours, suggesting they're gaming the system.


In [ ]:
USE AdventureWorks2022;

WITH ProdEmployees AS (
    SELECT 
        e.BusinessEntityID,
        p.FirstName,
        p.LastName,
        d.Name AS DepartmentName,
        e.JobTitle,
        e.VacationHours,
        e.SickLeaveHours,
        eph.Rate
    FROM HumanResources.Employee AS e
    JOIN HumanResources.EmployeeDepartmentHistory AS edh 
         ON e.BusinessEntityID = edh.BusinessEntityID
    JOIN HumanResources.Department AS d 
         ON edh.DepartmentID = d.DepartmentID
    JOIN Person.Person AS p 
         ON e.BusinessEntityID = p.BusinessEntityID
    JOIN HumanResources.EmployeePayHistory AS eph 
         ON e.BusinessEntityID = eph.BusinessEntityID
    WHERE d.Name = 'Production'
)
SELECT TOP (1)
    BusinessEntityID,
    FirstName,
    LastName,
    JobTitle,
    DepartmentName,
    Rate,
    VacationHours,
    SickLeaveHours
FROM ProdEmployees
ORDER BY VacationHours DESC, SickLeaveHours DESC;

### Case #02: The Stagnant Star
### The top-performing salesperson (by SalesYTD) hasn't received a commission or a pay raise in three years.
#### Objectives:
* Find the SalesPersonID with the highest SalesYTD in Sales.SalesPerson.
* Get their BusinessEntityID.
* Check their HumanResources.EmployeePayHistory and find the ModifiedDate of their last pay Rate change.


In [ ]:
-- Find top SalesYTD
SELECT TOP 1 BusinessEntityID, SalesYTD
FROM Sales.SalesPerson
ORDER BY SalesYTD DESC;

-- lists all salespeople in the company
SELECT sp.BusinessEntityID, p.FirstName, p.LastName, sp.SalesYTD
FROM Sales.SalesPerson sp
JOIN Person.Person p 
	ON sp.BusinessEntityID = p.BusinessEntityID;

-- finds the top-performing salesperson
SELECT p.FirstName, p.LastName, sp.SalesYTD, eph.Rate, eph.ModifiedDate
FROM Sales.SalesPerson sp
JOIN Person.Person p 
	ON sp.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.EmployeePayHistory eph 
	ON sp.BusinessEntityID = eph.BusinessEntityID
ORDER BY sp.SalesYTD DESC, eph.ModifiedDate DESC;

### Case #03: The Anonymous Tipper
### We received an email complaint about "shady dealings" from p.anderson@adventure-works.com. We need to find every employee this could belong to.
#### Objectives:
* Query the Person.Person table for FirstName starting with 'P' and LastName 'Anderson'.
* Get the BusinessEntityID for all matches.
* Check if these IDs exist in the HumanResources.Employee table to confirm they are current employees.


In [ ]:
-- Find matching people
SELECT BusinessEntityID, FirstName, LastName
FROM Person.Person
WHERE FirstName LIKE 'C%' AND LastName='Anderson';

-- Check if they’re employees
SELECT p.BusinessEntityID, e.BusinessEntityID AS EmpID
FROM Person.Person p
LEFT JOIN HumanResources.Employee e 
	ON p.BusinessEntityID=e.BusinessEntityID
WHERE p.FirstName LIKE 'C%' AND p.LastName='Anderson';

-- Show employment status
SELECT p.BusinessEntityID, p.FirstName, p.LastName,
CASE WHEN e.BusinessEntityID IS NOT NULL THEN 'Current Employee'
     ELSE 'Not an Employee' END AS EmploymentStatus
FROM Person.Person p
LEFT JOIN HumanResources.Employee e 
	ON p.BusinessEntityID=e.BusinessEntityID
WHERE p.FirstName LIKE 'C%' AND p.LastName='Anderson';

### Case #04 – The Seasonal Surge
### Sales managers want to verify which months produce the highest order totals across all years.
#### Objectives:
* Use Sales.SalesOrderHeader and extract the month name from OrderDate.
* Calculate total revenue (SUM(TotalDue)) per month.
* Rank months by sales volume and highlight the top 3.
* Bonus: Visualize using a simple monthly ranking table.

In [ ]:
USE AdventureWorks2022;

-- Step 1: Monthly totals across all years
WITH MonthlySales AS (
  SELECT
    DATENAME(MONTH, OrderDate) AS MonthName,
    MONTH(OrderDate)          AS MonthNum,
    SUM(TotalDue)             AS TotalSales
  FROM Sales.SalesOrderHeader
  GROUP BY DATENAME(MONTH, OrderDate), MONTH(OrderDate)
),

-- Step 2: Rank via self-join (count how many months have a higher total)
Ranked AS (
  SELECT
    m.MonthName, m.MonthNum, m.TotalSales,
    1 + COUNT(DISTINCT m2.TotalSales) AS SalesRank
  FROM MonthlySales m
  LEFT JOIN MonthlySales m2
    ON m2.TotalSales > m.TotalSales
  GROUP BY m.MonthName, m.MonthNum, m.TotalSales
)

-- Final: show all months, flag Top 3
SELECT
  MonthName,
  TotalSales,
  SalesRank,
  CASE WHEN SalesRank <= 3 THEN 'Top 3' ELSE '' END AS Remark
FROM Ranked
ORDER BY MonthNum;

### Case #05 – The Sluggish Shipper
### Customers are complaining that deliveries take far longer than promised, especially for large orders.
#### Objectives:
* Query Sales.SalesOrderHeader and calculate the difference in days between OrderDate and ShipDate.
* Find orders where the delay exceeds 10 days.
* Group by ShipMethodID to identify which shipping method is the slowest on average.
* Reveal the top 3 ShipMethod names responsible for the worst delays.


In [ ]:
USE AdventureWorks2022;
-- Step 1: Compute Delivery Delay per Order
WITH OrderDelays AS (
    SELECT
        soh.SalesOrderID,
        soh.ShipMethodID,
        soh.OrderDate,
        soh.ShipDate,
        DATEDIFF(DAY, soh.OrderDate, soh.ShipDate) AS DelayDays
    FROM Sales.SalesOrderHeader AS soh
    WHERE soh.ShipDate IS NOT NULL     -- ensure valid shipped orders
),

-- Step 2: Filter Orders with Noticeable Delay (> 2 days)
--   NOTE: changed from >10 to >2 to ensure visible output
DelayedOrders AS (
    SELECT 
        od.SalesOrderID,
        od.ShipMethodID,
        od.DelayDays
    FROM OrderDelays AS od
    WHERE od.DelayDays > 2
),

-- Step 3: Calculate Average Delay per ShipMethod
AvgDelayByMethod AS (
    SELECT 
        d.ShipMethodID,
        sm.Name AS ShipMethodName,
        COUNT(d.SalesOrderID) AS NumDelayedOrders,
        AVG(CAST(d.DelayDays AS DECIMAL(10,2))) AS AvgDelayDays
    FROM DelayedOrders AS d
    JOIN Purchasing.ShipMethod AS sm
        ON d.ShipMethodID = sm.ShipMethodID
    GROUP BY d.ShipMethodID, sm.Name
)

-- Final Step: Display Top 3 Slowest Shippers
SELECT TOP 3
    ShipMethodID,
    ShipMethodName,
    NumDelayedOrders,
    ROUND(AvgDelayDays, 2) AS AvgDelayDays
FROM AvgDelayByMethod
ORDER BY AvgDelayDays DESC;

### Case #06: The 'Special Pair'
### Both the customer and the employee received a 40% discount and live within Washington, United States.
#### Objectives:
* Find the BusinessEntityID of Employees in HumanResources.Employee.
* Find all customerID in Sales.Customer.
* Check if any customer matches with any employee where both have 40% discount

In [ ]:
-- Washington employees & customers
SELECT e.BusinessEntityID EmpID, c.CustomerID, a.City
FROM HumanResources.Employee e
JOIN Sales.Customer c 
	ON c.CustomerID > 0
JOIN Person.Address a 
	ON a.City = 'Seattle';  -- Washington example

-- Both got 40% discount
SELECT DISTINCT 
    soh.SalesOrderID,
    pEmp.FirstName AS Emp,
    pCust.FirstName AS Cust,
    sod.UnitPriceDiscount AS DiscountPct
FROM Sales.SalesOrderHeader AS soh
JOIN Sales.SalesPerson AS sp 
    ON soh.SalesPersonID = sp.BusinessEntityID
JOIN Sales.Customer AS c 
    ON soh.CustomerID = c.CustomerID
JOIN Person.Person AS pEmp 
    ON sp.BusinessEntityID = pEmp.BusinessEntityID
JOIN Person.Person AS pCust 
    ON c.PersonID = pCust.BusinessEntityID
JOIN Sales.SalesOrderDetail AS sod 
    ON soh.SalesOrderID = sod.SalesOrderID
WHERE sod.UnitPriceDiscount = 0.40;


### Case #07: The Cluster
### We've had a sudden spike in new customer signups, and they all live on the same street in Paris, France.
#### Objectives:
* Query Person.Address for City = 'Paris' and CountryRegion = 'France'.
* Group by AddressLine1 and count the number of distinct customers on each street.
* Identify the street(s) with a suspicious number of new accounts (>1).
* List each CustomerID that belongs to those clustered streets.

In [ ]:
-- Find clustered Paris streets
SELECT a.AddressLine1 Street, COUNT(DISTINCT c.CustomerID) NumCust
FROM Sales.Customer c
JOIN Person.Address a 
	ON c.CustomerID>0
JOIN Person.StateProvince sp 
	ON a.StateProvinceID=sp.StateProvinceID
JOIN Person.CountryRegion cr 
	ON sp.CountryRegionCode=cr.CountryRegionCode
WHERE a.City ='Paris' AND cr.Name ='France'
GROUP BY a.AddressLine1
HAVING COUNT(DISTINCT c.CustomerID)>1;

-- List customers on those streets
SELECT c.CustomerID, a.AddressLine1 Street, a.City
FROM Sales.Customer c
JOIN Person.Address a 
	ON c.CustomerID>0
WHERE a.City='Paris' AND a.AddressLine1 IN (
  SELECT a.AddressLine1
  FROM Person.Address a
  JOIN Person.StateProvince sp 
  	ON a.StateProvinceID=sp.StateProvinceID
  JOIN Person.CountryRegion cr 
  	ON sp.CountryRegionCode=cr.CountryRegionCode
  WHERE a.City='Paris' AND cr.Name='France'
  GROUP BY a.AddressLine1 HAVING COUNT(*) > 1
);